In [ ]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import $ivy.`org.apache.spark::spark-streaming-kafka-0-10:2.4.5`
import $ivy.`org.apache.spark::spark-sql-kafka-0-10:2.4.5`

In [ ]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql.streaming.Trigger
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SparkSession

def killAll() = {
    SparkSession
        .active
        .streams
        .active
        .foreach { x =>
                    val desc = x.lastProgress.sources.head.description
                    x.stop
                    println(s"Stopped ${desc}")
        }               
}

In [ ]:
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .config("spark.executor.instances", "2")
    .config("spark.executor.memory", "2g")
    .getOrCreate()
}

import spark.implicits._

In [ ]:
val df_content_test = spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "172.27.1.16:9092")
  .option("subscribe", "content_topic_example")
  .option("enable.auto.commit", "false")
  .option("auto.offset.reset","earliest")
  .option("startingOffsets", "earliest")
//   .option("auto.offset.reset","latest")
//   .option("endingOffsets", "latest")
//   .option("maxOffsetsPerTrigger", "10")
  .load()
  .select($"timestamp" cast "timestamp" as "contentTime" , $"key" cast "string" as "objID", $"value" cast "string")

def createConsoleSink(df: DataFrame) = {
    df
    .writeStream
    .format("console")
    .trigger(Trigger.ProcessingTime("2 seconds"))
    .option("truncate", "false")
    .option("numRows", "10")
}

val sink = createConsoleSink(df_content_test)
val sq = sink.start()

In [ ]:
sq.awaitTermination()

In [ ]:
killAll()

cmd0.sc:1: not found: value killAll
val res0 = killAll()
           ^Compilation Failed

: 

In [ ]:
spark.catalog.clearCache()
spark.close()
spark.stop()